<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/de_validations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re

def csv_files(path):
    """Função usada para ler os ficheiros que tem o simbolo do Euro"""
    import csv
    f = open(path, encoding='windows-1252', errors='ignore')
    data = []
    for row in csv.reader(f, delimiter=','):
        data.append(row)
    col = [*data[0]]
    data.pop(0)
    df = pd.DataFrame(data, columns=col)
    return df, col

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist
  

Check TowerDB Files

In [2]:
tw_index = 'code'
tw_doer = 'date_of_equipment_removal'
tw_status = 'site status (tims)'
tw_bts = 'bts_sites'
tw_bill = 'rfai ( ready for active installation ) '
tw_wip = 'wip_sites'
tw_decom = 'decommissioned site'
tw_critical = 'criticalsite_beyond_10'

msa_index = 'code'
msa_doer = 'date_of_equipment_removal'
msa_status = 'site status (tims)'
msa_bts = 'bts_sites'
msa_bill = 'rfai ( ready for active installation ) '
msa_wip = 'wip_sites'
msa_decom = 'decommissioned site'
msa_critical = 'criticalsite_beyond_10'
bill_col = lower_str(["Code","Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)",\
             "Categorization by Site Type",\
             "Categorization by Strategic Site","Categorization by Critical Site","Site Cluster",\
             "Strategic Site Bucket","Bts_sites","CriticalSite_Beyond_10","Sites As Metered Estimated","Active Sharing Arrangement",\
             "Indoor_Site_Any_Climate_Control","Outdoor_Site_With_Dc_Power","Rfai_Date","decommissioned site",\
             "Subsequent_Active_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Arrangement_Start_Date",\
             "First_Active_Sharing_Arrangement_End_Date"])


In [6]:
path_excel = '/content/Output - Skylon - TowerDB - Germany - Release6.3 20210726_Billing_ReducedScope_sharedwithCelfinet_v03 (1).xlsx'
#towerdb = pd.read_csv(path_towerdb,encoding='latin')
guideline = lower_str(['Code','Site Name','Macro Region','Region','Province','Municipality','Inhabitants','Address','Latitude','Longitude',\
             'Categorization by Transmission Sys','Categorization by Transmission Sys (sub-cluster)','Categorization by Site Type',\
             'Categorization by Indoor / Outdoor Site (Antennas)','Additional Site Type information','Categorization by Inhabitants',\
             'Rural/ Suburban/ Urban','Categorization by CONNECTIVITY','Technology VOD','POD ID','Energy Consumption current FY',\
             'Actual Energy Cost current FY','Infrastructure ready (existing)/ to be ready (new)','Infrastructure to be shared by',\
             'Counterpart','# of Lease Contracts','Current annual lease fees','Current energy lease fees','Current annual other fees',\
             'Total Annual Lease','(Average) residual duration','Maturity Cluster','ExCo rep. Avg Annual Lease costs',\
             'Total Energy Cost currrent FY (Energy provider + LL)','VOD (y/n)','Deutsche Telekom','Annual Fee per Tenant MNO1',\
             'Annual Energy Fee MNO1','Annual Maintenance Fee MNO1','Other Services Fee MNO1','Total Hosting Fee & Services MNO1',\
             'Residual duration MNO1','Maturity Clusters MNO1','Telefonica','Annual Fee per Tenant MNO2','Annual Energy Fee MNO2',\
             'Annual Maintenance Fee MNO2','Other Services Fee MNO2','Total Hosting Fee & Services MNO2','Residual duration MNO2',\
             'Maturity Clusters MNO2','E-Plus','Annual Fee per Tenant MNO3','Annual Energy Fee MNO3','Annual Maintenance Fee MNO3',\
             'Other Services Fee MNO3','Total Hosting Fee & Services','Residual duration MNO3','Maturity Clusters','# of OTMOs',\
             'Annual Fee from OTMOs','Annual Energy Fee from OTMOs','Annual Maintenance Fee OTMOs','Other Services Fee OTMOs',\
             'Total Hosting Fee & Services OTMOs','Average residual duration OTMOs','Maturity Clusters OTMOs','Total # of 3rd Party Tenants',\
             'Annual Fee from 3rd Party Tenants','Annual Energy Fee from 3rd Party Tenants','Annual Maintenance Fee from 3rd Party Tenants',\
             'Other Services Fee from 3rd Party Tenants','Total Hosting Fee & Services from 3rd Party Tenants',\
             'Weighted Average residual duration','Macro Cluster Tenancy','Macro Cluster Lease / Freeholds & Surface Right','Macro Cluster 1',\
             'Sites w/ at least a DDS (Lease Contract Type)','# of Tenants','Categorization by Tenant combination',\
             'Categorization by Type of Passive contracts','Categorization by Land/Surface ownership','In/Out',\
             'No of 3rd Party Tenants (PoP count)','Top cities classification','Type of Structure','Categorization by shared usage (contractual)',\
             'Categorization by Shared Usage (PoP count)','Security Class','Categorization by Sites with Single/Multiple Tower(s)',\
             'Categorization by Special Site/PLC','Categorization by Chimney-location','Categorization by Antenna Constraint',\
             'Categorization by 5G Roll-out Site','Categorization by Authority Site','Categorization by Transport-Sammler',\
             'Categorization by Strategic Site','Categorization by Dual Use Site','Categorization by Sublease Right','GBT Reserve Space','State',\
             'Postal code','Tower Height','Floor Space','Ground Register .1','Ground Register .2','Ground Register .3',\
             'Categorization by Critical Site','Site Cluster','infrastructure to be dismantled by','decommissioned site',\
             'Strategic Site Bucket','Bts_sites','CriticalSite_Beyond_10','Sites As Metered Estimated','Active Sharing Arrangement',\
             'Subsequent_Active_Sharing_Arrangement','First_Active_Sharing_Deployment_Type','First_Active_Sharing_Arrangement_Start_Date',\
             'First_Active_Sharing_Arrangement_End_Date','Indoor_Site_Any_Climate_Control','Outdoor_Site_With_Dc_Power','Rfai_Date',\
             'Cluster for non-enterprise DAS Sites','Site Status (TIMS)','Date_Of_Equipment_Removal'])

towerdb = pd.read_excel(path_excel, sheet_name='TowerDB', skiprows=12)
towerdb = towerdb.iloc[:, 2:]
towerdb.columns = lower_str(list(towerdb.columns))

path_msa = '/content/TowerDB_Germany_20210731.csv'
msa = pd.read_csv(path_msa,encoding='latin')
msa.columns = lower_str(list(msa.columns))

columns_to_merge = ['code', 'annual maintenance fee mno3', 'categorization by critical site', 'e-plus', 'categorization by antenna constraint', 'ground register .3', 'criticalsite_beyond_10', 'energy consumption current fy', 'first_active_sharing_deployment_type', 'indoor_site_any_climate_control', 'total hosting fee & services', 'subsequent_active_sharing_arrangement', 'categorization by indoor / outdoor site (antennas)', 'outdoor_site_with_dc_power', 'additional site type information', 'annual fee per tenant mno3', 'total energy cost currrent fy (energy provider + ll)', 'rfai_date', 'cluster for non-enterprise das sites', 'residual duration mno3', 'other services fee mno3', 'actual energy cost current fy', 'annual energy fee mno3', 'ground register .2', 'first_active_sharing_arrangement_start_date', 'ground register .1', 'first_active_sharing_arrangement_end_date', 'maturity clusters']
df_merge = msa[columns_to_merge]
towerdb = pd.merge(towerdb, df_merge, how='left', on='code')

towerdb = towerdb[guideline]

dates_tw = ['first_active_sharing_arrangement_start_date','first_active_sharing_arrangement_end_date', 'date_of_equipment_removal']
for i in dates_tw:
    towerdb[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in towerdb[i]]

ints = ['actual energy cost current fy', 'counterpart', '# of lease contracts', 'current energy lease fees',\
        'total annual lease', 'deutsche telekom', 'annual energy fee mno1', 'total hosting fee & services mno1', 'telefonica',\
        'annual energy fee mno2', '# of otmos', 'annual energy fee from otmos', 'total hosting fee & services otmos',\
        'total # of 3rd party tenants', 'annual energy fee from 3rd party tenants', 'annual maintenance fee from 3rd party tenants',\
        'other services fee from 3rd party tenants', 'total hosting fee & services from 3rd party tenants',\
        'no of 3rd party tenants (pop count)', 'security class']
for i in ints:
    towerdb[i] = towerdb[i].fillna(0)
    towerdb[i] = list(map(int, towerdb[i]))

bill_col = lower_str(["Code","Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)",\
             "Categorization by Site Type",\
             "Categorization by Strategic Site","Categorization by Critical Site","Site Cluster",\
             "Strategic Site Bucket","Bts_sites","CriticalSite_Beyond_10","Sites As Metered Estimated","Active Sharing Arrangement",\
             "Indoor_Site_Any_Climate_Control","Outdoor_Site_With_Dc_Power","Rfai_Date","decommissioned site",\
             "Subsequent_Active_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Arrangement_Start_Date",\
             "First_Active_Sharing_Arrangement_End_Date"])

for i in bill_col:
    towerdb[i] = towerdb[i].replace(['N/A', 'n/a',"0", 0, '-'], '')


# CSV Reading

In [ ]:
path_towerdb = '/content/TowerDB_Germany_20210731.csv'

towerdb = pd.read_csv(path_towerdb,encoding='latin')
towerdb.columns = lower_str(list(towerdb.columns))

dates_tw = ['first_active_sharing_arrangement_start_date','first_active_sharing_arrangement_end_date', 'date_of_equipment_removal']
for i in dates_tw:
    towerdb[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in towerdb[i]]

ints = ['security class', 'postal code']
for i in ints:
    towerdb[i] = towerdb[i].fillna(0)
    towerdb[i] = list(map(int, towerdb[i]))

#towerdb = towerdb.replace(['N/A', 'n/a',"0", '-', , np.nan,'nan'], '')
towerdb = towerdb.fillna('')

bill_col = lower_str(bill_col)
for i in bill_col:
    towerdb[i] = towerdb[i].replace(['N/A', 'n/a',"0", 0, '-'], '')

"""Read MSA File"""
path_old = '/content/TowerDB_Germany_20210630.csv'
msa = pd.read_csv(path_old, encoding='latin1')
msa_cols = lower_str(list(msa.columns))
msa.columns = msa_cols

towerdb = towerdb[msa_cols]

In [ ]:
sites = [i for i in towerdb[towerdb['indoor_site_any_climate_control']=='Yes; Indoor Air Conditioning and Free Air cooling / Free cooling units']['site code']]
towerdb.loc[towerdb['site code'].isin(sites), 'climate control (yes/no)'] = 'Yes; Indoor Air Conditioning'

sites = [i for i in towerdb[towerdb['climate control (yes/no)']=='Yes; Indoor Free Air cooling / Free cooling units']['site code']]
towerdb.loc[towerdb['site code'].isin(sites), 'climate control (yes/no)'] = 'Yes; Indoor Air Conditioning'

In [ ]:
"""First Step Check All columns received"""
def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

#check billing columns
df_col_missing = check_columns_received(towerdb, msa_cols)
df_col_missing
#tem error Falta Columns de Billing trigger

,Column(s) Missing


In [7]:
def check_date_columns(df, df_index,status_col,columns):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_dates = df[columns].fillna('')
    df_dates = df_dates.to_dict(orient='list')
    #df_dates['sites'] = df_dates[df_index]
    #df_dates = df_dates.set_index('sites')
    new_dic = defaultdict(list)

    date_format = re.compile(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}")

    for column in set(df_dates.keys()):
        for value in df_dates[column]:
            if date_format.match(value) == None:
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = df[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, status_col]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[status_col]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors

dates_cols = ['Date_Of_Equipment_Removal']
dates_cols = lower_str(dates_cols)
#Check dates of column 'Date_Of_Equipment_Removal' Pg 7
#check_date_columns(df, df_index,status_col,columns, format)
dftw_dates_errors = check_date_columns(towerdb, tw_index, tw_status, dates_cols) 
dftw_dates_errors

,site status (tims),date_of_equipment_removal
code,,
0001 B,In Service,Blank Value
0001 D,In Service,Blank Value
0001 M,In Service,Blank Value
0001 O,In Service,Blank Value
0001 S,In Service,Blank Value
...,...,...
Y157 M,In Service,Blank Value
Y158 M,In Service,Blank Value
Y159 M,In Service,Blank Value


In [ ]:
towerdb[towerdb[tw_index]=='0001 B']['categorization by transmission sys']

0    Macro
Name: categorization by transmission sys, dtype: object

In [ ]:
"""Check all picklist Columns"""
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors = df_errors.dropna(how='all', axis=1)
    
    if not df_errors.empty:
        #df = df[[df_index, df_status]]
        #df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors[df_index] = df_picklist[df_index]
        df_errors[df_status] = df_picklist[df_status]
        df_errors = df_errors.set_index(df_index)
        df_errors = df_errors.dropna(how='all', axis=0)
        df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
        df_errors = df_errors.reset_index()
        return df_errors
    else:
        print('\nNo one Picklist Error Founded!\n')

picklist_col = ["Code",'site status (tims)','Categorization by Transmission Sys','Categorization by Transmission Sys (sub-cluster)',\
            'Categorization by Site Type', 'Categorization by Indoor / Outdoor Site (Antennas)',\
            'Categorization by Strategic Site','Categorization by Critical Site', 'Strategic Site Bucket', 'Bts_sites',\
            'CriticalSite_Beyond_10','decommissioned site', 'Sites As Metered Estimated',\
            'Active Sharing Arrangement', 'Indoor_Site_Any_Climate_Control', 'Outdoor_Site_With_Dc_Power']
picklist_col = lower_str(picklist_col)
picklist = {
    'categorization by transmission sys': ['Macro','Public DAS Sites','Repeater Sites','Transmission', 'Long-Term Mobile Sites', 'Non-Enterprise DAS'],
    'categorization by transmission sys (sub-cluster)': ['Core', 'DAS', 'Long-term Mobile', 'Non-Enterprise DAS', 'Repeater', 'Standard', 'Transmission'],
    'categorization by site type':['DAS','GBT','RTT', 'Non-Enterprise DAS'],
    'categorization by indoor / outdoor site (antennas)': ['Outdoor', 'Indoor'],
    'categorization by strategic site':['Yes','No'],
    'categorization by critical site':['Yes','No'],
    'strategic site bucket': ['Yes - 0-5%','Non Strategic'],
    'bts_sites': ['Yes','No'],
    'criticalsite_beyond_10': ['Beyond 10%','Within 10%','Non Critical'],
    'decommissioned site': ['Yes', 'No'],
    'sites as metered estimated': ['Estimated Model','Metered Model'],
    'active sharing arrangement': ['No Active Sharing'],
    'indoor_site_any_climate_control': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Free Air cooling / Free cooling units'],
    'outdoor_site_with_dc_power':['AC','DC','AC & DC','No Power']}
actives = towerdb[towerdb[tw_status]=='In Service']
df_picklist_errors = check_picklist_v1(actives,'code','site status (tims)',picklist_col, picklist)
df_picklist_general = check_picklist_v1(towerdb,'code','site status (tims)',picklist_col, picklist)
print(df_picklist_errors)
print('\n')
print(df_picklist_general)


Falta UIS para gerar todos os Checks


In [ ]:
#Read UIP File
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel('/content/UserInput_Germany_20210831_InMonth.xlsx',sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip.columns = uip_names

msa_sites = [i for i in msa[msa[msa_status]=='In Service']['code']]
tw_sites = [i for i in towerdb[towerdb[tw_status]=='In Service']['code']]
uip_sites = [i for i in uip['Site_ID']]


In [ ]:
"""Eigth Check for BTS sites"""
def check_bts(df_tw, bts_tw_columns, tw_index, status_col, df_msa, bts_msa_column, msa_index):

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    filtered = df_tw[[tw_index, status_col, bts_tw_columns]]
    bts_out_tw = [i for i in tw_bts_sites if i not in bts_msa]
    df = pd.DataFrame(bts_out_tw, columns=['BTS MSA Out TowerDB'])
    df = pd.merge(df, filtered, how='left', left_on=['BTS MSA Out TowerDB'], right_on=tw_index)
    df = df[['BTS MSA Out TowerDB', status_col, bts_tw_columns]]
    return df

# Check for BTS sites
df_bts_out_tw = check_bts(towerdb,tw_bts,tw_index,tw_status,msa,msa_bts,msa_index)
df_bts_out_tw


In [ ]:
"""Nineth - Check fopr decomissioned sites"""
def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    #c = country.lower()
    sites = [i for i in df[df[decom_col]=='Yes'][df_index]]

    filtered = df[(df[df_index].isin(sites))&(df[doer_col]=="")]
    return filtered[[df_index,status_col, decom_col, doer_col]]

#Validate Decomissioned Sites
df_decom_errors = check_decommissioned(towerdb,tw_index, tw_status, 'decommissioned site', tw_doer)
df_decom_errors
#None errors

In [ ]:
"""Tenth - Check Date Of Equipement Removed date for in service sites"""
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    df_tw = df_tw[df_tw[status_col]==status][[tw_index, status_col, date_col]] 
    dates = [i for i in df_tw[date_col] if i!= '']

    if t == 'doer':
        filtered = df_tw[df_tw[date_col].astype('datetime64[ns]') < current_date]
        return filtered
    else:
        #bill_dates = pd.to_datetime(df_tw[date_col], errors==coerrce)
        filtered = df_tw[df_tw[date_col]=='']
        return filtered

df_doer_errors = check_tw_bill_doer(towerdb, tw_index, tw_doer , tw_status,'In Service', 'doer')
df_doer_errors
# None Errors

In [ ]:
"""Eleventh - Check Month-on-Month towerdb sites"""
def check_mom_bts(df_tw, tw_index,status_col,df_bts, df_msa, msa_index, msa_bts):
 
    msa_bts_sites = [i for i in df_msa[df_msa[msa_bts]=='Yes'][msa_index]]

    tw_bts_sites = [i for i in df_tw[df_tw[df_bts]=='Yes'][tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]

    filtered = df_msa.loc[df_msa[tw_index].isin(out_tower_bts)]

    return filtered[[tw_index,status_col, df_bts]]    

df_mom_errors = check_mom_bts(towerdb,tw_index, tw_status, tw_bts, msa,msa_index, msa_bts)
df_mom_errors
# None Errors

In [ ]:
"""Twelveth - Match UIS sites with Towerdb Sites """
def check_uip_tw(df_tw,tw_index, status_tw_col,decom_col, tw_bts_col,tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col, tw_bts_col, decom_col]]
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]

    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    
    #Check Decomissioned Sites
    tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col, decom_col]]

    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]   
                           
    #if not set(bts_sites).intersection(uip_sites):
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col, tw_bts_col]]

    bts_tw_critical = [i for i in df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]]
    critical = [i for i in bts_tw_critical if i not in uip_critical]
    critical = pd.DataFrame(critical, columns=['Sites Beyond 10% out UIS'])
    critical = pd.merge(critical, filtered, how='left', left_on='Sites Beyond 10% out UIS',\
                                    right_on=tw_index)
    critical = critical[['Sites Beyond 10% out UIS', status_tw_col]]

    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis,decomiss_sites_in_uip, bts_sites_out_uip, critical

uis_sites_not_in_towerdb, in_service_not_in_uis,decomiss_sites_in_uip,  bts_sites_out_uip, critical = check_uip_tw(towerdb, tw_index, tw_status,tw_decom, tw_bts, \
                                                                                            tw_critical, uip, uip_sites)
print(uis_sites_not_in_towerdb)
print('\n')
print(in_service_not_in_uis)
print('\n')
print(decomiss_sites_in_uip)
print('\n')
print(bts_sites_out_uip)
print('\n')
print(critical)
#None Errors


Commercial

In [ ]:
act = '/content/UserInput_Germany_20210831_InMonth.xlsx'
old = '/content/UserInput_Germany_20210731_InMonth.xlsx'
cols_ordered = ['Charge_Type','Sub_charge_Type', 'Param1','Param2', 'Data_Type','Input_Value_actual', 'Input_Value_before',\
                'Equal Values','Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_charge_Type', 'Param1','Param2', 'Data_Type', 'Description/Instruction', 'Frequency of Update']
name = ['Charge_Type','Sub_charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value', 'Description/Instruction', 'Frequency of Update'] 
col = ['Input_Value']               
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna(0, inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names).fillna('')
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names ).fillna('')
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols).fillna('')

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo errors founded!')

df_comm = check_commercial(act, old, col, name, merge_cols, cols_ordered)
df_comm




No errors founded!


In [ ]:
def check_diffs_v2(path_current, path_last):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)

    _actual = pd.read_excel(path_current,sheet_name='Commercial').fillna('')

    _before = pd.read_excel(path_last,sheet_name='Commercial').fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    #df_final.style.apply(highlight_diff, axis=None)
    if not df_final.empty:
        return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
    else:
        print('\nNo differences Founded!\n')

df_com_vf = check_diffs_v2(act, old)
df_com_vf

Generate xlsx Log with invalid or incorrect values founded in TowerDB File

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0])
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0])

    writer.save() 

log_dfs = [['DOER Dates Errors', dftw_dates_errors],
           ['General Picklist Errors', df_picklist_general],
           ['Actives Picklist Errors', df_picklist_errors],
           ['Decom Sites Date Error', df_decom_errors],
           ['DOER Dates Errors', df_doer_errors],
           ['MSA BTS not in TowerDB', df_mom_errors],
           ['UIS Sites not active in TowerDB', uis_sites_not_in_towerdb],
           ['TowerDB Sites out of UIS', in_service_not_in_uis],
           ['Decom Sites In UIS', decomiss_sites_in_uip],
           ['UIS BTS not in TowerDB(BTS)', bts_sites_out_uip],
           ['Sites Beyond 10% out UIS', critical],
           ['Commercial Differences',df_comm]]

path_errors = '/content/DE_TowerDB_Errors.xlsx'
general_log_erros(log_dfs, path_errors)

TA Input File Check

Special Case(Excel File)

In [ ]:
path_in_month = '/content/Output - Skylon - TowerDB - Germany - Release6.3 20210726_Billing_ReducedScope_sharedwithCelfinet_v03 (1).xlsx'

#towerdb[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in towerdb[i]]
ta = pd.read_excel(path_in_month, sheet_name='TA_Input', skiprows=1)

dates_ta = ['Starting date','Expiring date', 'Expiring date after renewal']
for i in dates_ta:
    ta[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in ta[i]]

ta = ta.rename(columns={'Contract Migration Scope': 'Contract Migration Scope 14.05.2020'})
ta.columns = lower_str(list(ta.columns))
#ta.columns = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), ta.columns.to_list()))
ta['tenant (id)'] = ta['tenant (id)'].fillna(0)
ta['tenant (id)'] = ta['tenant (id)'].astype(int)

guideline_ta = lower_str(['Code',\
          'Tenant Agreement',
          'Tenant',\
          'Tenant Cluster',\
          'MNO Cluster',\
          'Tenant (ID)',
          'Starting date','Expiring date',\
          'Annual Fee per Tenant',
          'Overlapping days in FY20',
          'Current Annual Fee per Tenant',\
          'Annual Energy Fee from 3rd Party Tenants',
          'OTP Amount',
          'Renewal option',
          'Residual duration',\
          'Maturity Cluster',
          'Contract Scope',
          'Site Scope',
          'Contract Migration Scope 14.05.2020',
          'Comment'])

ta = ta[guideline_ta]

ta.head(3)

In [ ]:
import pandas as pd
#Read TA Input File
path_ta_last = '/content/TA_Input_Germany_20210731.csv'

ta_col = ['Code',\
          'Tenant Agreement',
          'Tenant',\
          'Tenant Cluster',\
          'MNO Cluster',\
          'Tenant (ID)',
          'Starting date','Expiring date',\
          ' Annual Fee per Tenant ',
          'Overlapping days in FY20',
          ' Current Annual Fee per Tenant ',\
          'Annual Energy Fee from 3rd Party Tenants',
          ' OTP Amount ',
          ' Renewal option ',
          ' Residual duration ',\
          'Maturity Cluster',
          'Contract Scope',
          'Site Scope',
          'Contract Migration Scope 14.05.2020',
          'Comment']

ta = pd.read_csv(path_ta_input, encoding='latin')

dates_ta = ['Starting date','Expiring date', 'Expiring date after renewal']
for i in dates_ta:
    ta[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in ta[i]]

ta = ta[ta_col]

ta_bill = ['Tenant Agreement','Tenant', 'Starting date','Expiring date']

for i in ta_bill:
    ta[i] = ta[i].replace(['N/A', 'n/a',"0", 0, '_',' ', np.nan,'nan'], '')

ta.to_csv('/content/TA_Input_Germany_20210831.csv', index=False)

ta.head(2)

In [ ]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):
    df[start_date] = pd.to_datetime(df[start_date], format='%d/%m/%Y')
    df[end_date] = pd.to_datetime(df[end_date] , format='%d/%m/%Y')
    filtered = df[df[start_date] > df[end_date]]
    return filtered[[tw_index, start_date,end_date]]

#Check date format(dd/mm/YYYY)
df_dates_erros = check_lc_ta_dates(ta, 'Code','Starting date', 'Expiring date')
df_dates_erros

In [ ]:
#Check Amount values
def check_amounts(df_check, df_index, columns, pattern=','):
    from collections import defaultdict

    df = df_check[columns].fillna('')
    df = df.to_dict(orient='list')

    new_dic = defaultdict(list)
    for id in df_check[df_index]:
        new_dic[df_index].append(id)

    for column in columns:
        for value in df[column]:
            value = str(value)
            if value == '':
                new_dic[column].append('Blank')
            elif not value.__contains__(pattern):
                new_dic[column].append(f'Incorrect format: {value}')
            else:
                new_dic[column].append("Ok")
    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.replace('Ok', np.nan)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    return df_errors

cols = lower_str(['Annual Fee per Tenant', 'Current Annual Fee per Tenant'])  
            
df = check_amounts(ta, 'code', cols,'.')
df

LC Input File Check

In [ ]:
path_lc = '/content/Output - Skylon - TowerDB - Germany - Release6.3 20210726_Billing_ReducedScope_sharedwithCelfinet_v03 (1).xlsx'

lc = pd.read_excel(path_lc, sheet_name='LC_Input', skiprows=1)

lc.head(3)

In [ ]:
#Read TA Input File
lc_cols = ['Lease Contract ID','Code','Counterpart','Counterpart Cluster','Counterpart (ID)','Starting date','Expiring date',\
           ' Annual lease fees ','Overlapping days in FY20',' Current annual lease fees ',' OTP Amount ',\
           'Renewal option','(Average) residual duration','Maturity Cluster','Site Scope','Contract scope','MNO Cluster','reason for scope',\
           'Adjustment','+/-','Contract Migration Scope 14.05.2020']

path_lc_input = '/content/LC_Input_Germany_20210731.csv'
lc = pd.read_csv(path_lc_input, encoding='latin')

lc_bill = ['Lease Contract ID', 'Starting date','Expiring date', ' Annual lease fees ']
for i in lc_bill:
    lc[i] = lc[i].replace(['N/A', 'n/a',"0", 0, '_',' ', np.nan,'nan'], '')
lc = lc[lc_cols]

lc.to_csv('/content/LC_Input_Germany_20210831.csv', index=False)
lc.head(1)


,Lease Contract ID,Code,Counterpart,Counterpart Cluster,Counterpart (ID),Starting date,Expiring date,Annual lease fees,Overlapping days in FY20,Current annual lease fees,OTP Amount,Renewal option,(Average) residual duration,Maturity Cluster,Site Scope,Contract scope,MNO Cluster,reason for scope,Adjustment,+/-,Contract Migration Scope 14.05.2020
0,1100002245/1100000642,0001 B,Gemeinn.Wohnungsbau AG Berlin,OTMO,800001944,05/12/1990,31/12/2030,8980.78,365,8980.7825,NaN,On-going,9.583333,<= 10 years,in scope,In Scope,OTMO,"condition, or indicator for one-time payment o...",NaN,NaN,In scope


In [ ]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):
    df[start_date] = pd.to_datetime(df[start_date], format='%d/%m/%Y')
    df[end_date] = pd.to_datetime(df[end_date] , format='%d/%m/%Y')
    filtered = df[df[start_date] > df[end_date]]
    return filtered[[tw_index, start_date,end_date]]


df_dates_erros = check_lc_ta_dates(lc, 'Code', 'Starting date', 'Expiring date')
df_dates_erros


,Code,Starting date,Expiring date


In [ ]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors = df_errors.dropna(how='all', axis=1)

In [ ]:
def check_amounts(df_check, df_index, columns, pattern=','):
    from collections import defaultdict

    df = df_check[columns].fillna('')
    df = df.to_dict(orient='list')

    new_dic = defaultdict(list)
    for id in df_check[df_index]:
        new_dic[df_index].append(id)

    for column in columns:
        for value in df[column]:
            value = str(value)
            if value == '':
                new_dic[column].append('Blank')
            elif not value.__contains__(pattern):
                new_dic[column].append(f'Incorrect format: {value}')
            else:
                new_dic[column].append("Ok")

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.replace('Ok', np.nan)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    return df_errors

lc_cols = ['Annual lease fees']
lc_am = check_amounts(lc, 'Code',lc_cols, '.')
lc_am

""
Code
